In [ ]:
%run common.ipynb

# Trials

In [ ]:
def trial(nn='CNN', N_range=[5,6], random_states=[0, 17]):
    print('Processing...')
    print()

    a_N = np.array([])
    a_sqrt_mse = np.array([])
    a_avg_abs_errors = np.array([])
    a_std_abs_errors = np.array([])

    # change N [32..2048]
    for i in N_range:

        N = 2**i

        a_N = np.append(a_N, N)

        # data generation (fix random state)
        DATA_RANDOM_STATE = 0
        fix_random(seed=DATA_RANDOM_STATE)
        X, y, M = generate_data(N=N, 
                                num_alphas=100, 
                                samples_per_alpha=100, 
                                random_alpha=True, 
                                max_M_only=False, 
                                random_state=DATA_RANDOM_STATE)

        # reshape X for Conv1D
        if 'CNN' == nn:
            X = X.reshape(X.shape[0], X.shape[1], 1)
        elif 'DNN2' == nn:
            X = data_hstack_windows_sum(X, window_sizes=None)

        # split train/test
        print('input.shape:', X.shape)
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=DATA_RANDOM_STATE)

        # create model
        if 'CNN' == nn:
            model = create_cnn_model(X.shape[1], filters=32)
        elif 'CNNM' == nn: # CNN-multi-layers
            model = create_cnn_model_multi_layer(X.shape[1], filters=32)
        elif nn.startswith('DNN'):
            model = create_dnn_model(X_train.shape[1], 
                            layers=[256,256], 
                            activation='relu', 
                            init=keras.initializers.HeUniform(seed=DATA_RANDOM_STATE), 
                            batch_normalization=True, 
                            dropout=0.0, 
                            optimizer=Adam(lr=1e-2), 
                            k_reg=True, 
                            k_reg_lr=1e-5, 
                            a_reg=True, 
                            a_reg_lr=1e-5)
        else:
            raise RuntimeError("nn: '{}' not supported".format(nn))

        # average sqrt_mse among different random states (fixed N)
        avg_sqrt_mse = 0

        # array of absolute errors
        avg_abs_errors = np.zeros_like(y_test)

        # change random state
        for rs in random_states:

            # fix random
            fix_random(seed=rs)
        
            # training
            model, history = train(model, X_train, y_train, batch_size=32, random_state=rs)

            # predict
            y_pred = model.predict(X_test).flatten()

            # mse
            mse = mean_squared_error(y_test, y_pred)

            # sqrt_mse
            sqrt_mse = np.sqrt(mse)

            # avg_sqrt_mse (accumulator)
            avg_sqrt_mse += sqrt_mse
            
            #print('N = {}, M = {}, random_state = {}    =>    sqrt_mse = {:.6f}'.format(N, M, rs, sqrt_mse))

            # absolute errors
            avg_abs_errors += np.abs(y_test - y_pred)

        # avg errors across random_states
        avg_abs_errors = avg_abs_errors / len(random_states)
        
        # avg errors across samples (t_test)
        a_avg_abs_errors = np.append(a_avg_abs_errors, np.average(avg_abs_errors))

        # std across samples
        a_std_abs_errors = np.append(a_std_abs_errors, np.std(avg_abs_errors))

        # avg_sqrt_mse
        avg_sqrt_mse = avg_sqrt_mse / len(random_states)
        a_sqrt_mse = np.append(a_sqrt_mse, avg_sqrt_mse)
        print('N: {} \t avg_sqrt_mse = {:.6f}'.format(N, avg_sqrt_mse))
        print()

    return a_N, a_sqrt_mse, a_avg_abs_errors, a_std_abs_errors

# CONSTANTS

In [ ]:
N_RANGE=[5,6,7]; RANDOM_STATES=[0, 17]
# N_RANGE=range(5, 12); RANDOM_STATES=[0, 3, 5, 8, 11, 16, 17, 20, 21, 24]

# DNN trials

In [ ]:
a_N, a_sqrt_mse_dnn, a_avg_abs_errors_dnn, a_std_abs_errors_dnn = trial(nn='DNN', N_range=N_RANGE, random_states=RANDOM_STATES)

# CNN trials

In [ ]:
a_N, a_sqrt_mse_cnn, a_avg_abs_errors_cnn, a_std_abs_errors_cnn = trial(nn='CNN', N_range=N_RANGE, random_states=RANDOM_STATES)

# CNN multi-layer trials

In [ ]:
# TODO
# a_N, a_sqrt_mse_cnn2, a_avg_abs_errors_cnn2, a_std_abs_errors_cnn2 = trial(nn='CNNM', N_range=N_RANGE, random_states=RANDOM_STATES)

# DNN trials with data augmentation (hstack windows sums of sizes [2,4,8,...])

In [ ]:
a_N, a_sqrt_mse_dnn2, a_avg_abs_errors_dnn2, a_std_abs_errors_dnn2 = trial(nn='DNN2', N_range=N_RANGE, random_states=RANDOM_STATES)

# plot log(N) vs sqrt_mse

In [ ]:
def plot_sqrt_mse():
    
    plt.figure(figsize=(15, 4))
    
    ax1 = plt.subplot(1, 2, 1)

    ax1.set(title='sqrt_mse', xlabel='log(N)', ylabel='sqrt_mse')

    x = np.log10(a_N)

    y = a_sqrt_mse_cnn
    std = a_std_abs_errors_cnn
    ax1.plot(x, y, '.b-', linewidth=2, markersize=10, label='cnn')
    ax1.fill_between(x, y - std, y + std, facecolor='blue', alpha=0.3)
    
    y = a_sqrt_mse_dnn
    std = a_std_abs_errors_dnn
    ax1.plot(x, y, '.r-', linewidth=2, markersize=10, label='dnn')
    ax1.fill_between(x, y - std, y + std, facecolor='red', alpha=0.3)
    
    plt.legend(loc='upper right')

plot_sqrt_mse()


# print statistics

In [ ]:
df = pd.DataFrame({
    'N': a_N, 
    'sqrt_mse(DNN)': a_sqrt_mse_dnn, 
    'sqrt_mse(CNN)': a_sqrt_mse_cnn, 
    'std(DNN)': a_std_abs_errors_dnn, 
    'std(CNN)': a_std_abs_errors_cnn, 
    'avg_abs_error(DNN)': a_avg_abs_errors_dnn, 
    'avg_abs_error(CNN)': a_avg_abs_errors_cnn, 
})


df = df.set_index('N')

columns=[
    ('sqrt(MSE)','DNN'), ('sqrt(MSE)','CNN'), 
    ('STD', 'DNN'), ('STD', 'CNN'),
    ('avg(abs(err))','DNN'), ('avg(abs(err))','CNN'), 
]

df.columns=pd.MultiIndex.from_tuples(columns)

print('err = y_test - y_pred')
df